In [1]:
import argparse
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=20, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
opt = parser.parse_known_args()[0]
print(opt)

Namespace(b1=0.5, b2=0.999, batch_size=64, channels=1, img_size=28, latent_dim=100, lr=0.0002, n_cpu=8, n_epochs=20, sample_interval=400)


In [3]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [4]:
generator.parameters()

<generator object Module.parameters at 0x7f81a7342888>

In [5]:
for epoch in range(1):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)
        print(gen_imgs)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        break

tensor([[[[-1.1322e-02,  2.5669e-02, -1.7390e-02,  ..., -1.8620e-02,
           -9.1094e-04, -5.3145e-03],
          [ 1.7015e-02, -9.6458e-03,  9.2328e-03,  ..., -4.7894e-03,
            3.7301e-03,  1.4213e-02],
          [-2.5357e-02, -1.2654e-02, -2.2538e-02,  ...,  1.5724e-02,
            2.3528e-02, -2.6456e-02],
          ...,
          [ 5.9817e-03,  4.6695e-03,  1.6687e-02,  ..., -1.0555e-02,
            3.3763e-02, -2.2458e-02],
          [-3.2602e-02,  1.6600e-02, -2.1343e-02,  ..., -5.7349e-03,
            2.6797e-02, -1.7579e-02],
          [-1.8965e-02,  1.2922e-02,  3.9895e-02,  ...,  2.4909e-02,
           -1.0044e-02, -1.2304e-02]]],


        [[[-7.0963e-03,  2.2651e-02, -2.2744e-02,  ..., -1.9436e-02,
            6.5569e-04,  5.1885e-04],
          [ 1.6725e-02, -8.2910e-03,  2.9611e-03,  ..., -6.8492e-03,
            6.0031e-03,  1.2435e-02],
          [-2.3566e-02, -8.7089e-03, -2.5032e-02,  ...,  1.2675e-02,
            2.0957e-02, -1.9578e-02],
          ...,
   

In [6]:
real_loss

tensor(0.7099, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [7]:
fake_loss

tensor(0.7042, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [8]:
d_loss = (real_loss+fake_loss)/2
d_loss

tensor(0.7071, device='cuda:0', grad_fn=<DivBackward0>)

In [9]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z);

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
    break

[Epoch 0/20] [Batch 0/938] [D loss: 0.626083] [G loss: 0.681743]
[Epoch 0/20] [Batch 1/938] [D loss: 0.563178] [G loss: 0.681599]
[Epoch 0/20] [Batch 2/938] [D loss: 0.515212] [G loss: 0.681673]
[Epoch 0/20] [Batch 3/938] [D loss: 0.475328] [G loss: 0.681581]
[Epoch 0/20] [Batch 4/938] [D loss: 0.437436] [G loss: 0.681404]
[Epoch 0/20] [Batch 5/938] [D loss: 0.412497] [G loss: 0.681053]
[Epoch 0/20] [Batch 6/938] [D loss: 0.394347] [G loss: 0.680502]
[Epoch 0/20] [Batch 7/938] [D loss: 0.382435] [G loss: 0.679600]
[Epoch 0/20] [Batch 8/938] [D loss: 0.374726] [G loss: 0.678466]
[Epoch 0/20] [Batch 9/938] [D loss: 0.370439] [G loss: 0.677129]
[Epoch 0/20] [Batch 10/938] [D loss: 0.367565] [G loss: 0.675216]
[Epoch 0/20] [Batch 11/938] [D loss: 0.366240] [G loss: 0.673176]
[Epoch 0/20] [Batch 12/938] [D loss: 0.366861] [G loss: 0.669600]
[Epoch 0/20] [Batch 13/938] [D loss: 0.367040] [G loss: 0.666676]
[Epoch 0/20] [Batch 14/938] [D loss: 0.369966] [G loss: 0.661990]
[Epoch 0/20] [Batch 

[Epoch 0/20] [Batch 125/938] [D loss: 0.401464] [G loss: 1.582079]
[Epoch 0/20] [Batch 126/938] [D loss: 0.412467] [G loss: 1.000926]
[Epoch 0/20] [Batch 127/938] [D loss: 0.309493] [G loss: 1.154400]
[Epoch 0/20] [Batch 128/938] [D loss: 0.351883] [G loss: 1.367617]
[Epoch 0/20] [Batch 129/938] [D loss: 0.377548] [G loss: 0.973392]
[Epoch 0/20] [Batch 130/938] [D loss: 0.273389] [G loss: 1.208139]
[Epoch 0/20] [Batch 131/938] [D loss: 0.307742] [G loss: 1.340296]
[Epoch 0/20] [Batch 132/938] [D loss: 0.364758] [G loss: 0.975728]
[Epoch 0/20] [Batch 133/938] [D loss: 0.317019] [G loss: 1.239547]
[Epoch 0/20] [Batch 134/938] [D loss: 0.373654] [G loss: 1.059658]
[Epoch 0/20] [Batch 135/938] [D loss: 0.394059] [G loss: 1.031342]
[Epoch 0/20] [Batch 136/938] [D loss: 0.362205] [G loss: 1.161908]
[Epoch 0/20] [Batch 137/938] [D loss: 0.366856] [G loss: 1.055215]
[Epoch 0/20] [Batch 138/938] [D loss: 0.355032] [G loss: 1.092398]
[Epoch 0/20] [Batch 139/938] [D loss: 0.380291] [G loss: 1.112

[Epoch 0/20] [Batch 255/938] [D loss: 0.624850] [G loss: 1.185298]
[Epoch 0/20] [Batch 256/938] [D loss: 0.658479] [G loss: 0.422919]
[Epoch 0/20] [Batch 257/938] [D loss: 0.594693] [G loss: 1.142087]
[Epoch 0/20] [Batch 258/938] [D loss: 0.557019] [G loss: 0.554604]
[Epoch 0/20] [Batch 259/938] [D loss: 0.513153] [G loss: 1.110351]
[Epoch 0/20] [Batch 260/938] [D loss: 0.525444] [G loss: 0.593935]
[Epoch 0/20] [Batch 261/938] [D loss: 0.510370] [G loss: 1.296798]
[Epoch 0/20] [Batch 262/938] [D loss: 0.550926] [G loss: 0.522038]
[Epoch 0/20] [Batch 263/938] [D loss: 0.520201] [G loss: 1.317882]
[Epoch 0/20] [Batch 264/938] [D loss: 0.558223] [G loss: 0.515485]
[Epoch 0/20] [Batch 265/938] [D loss: 0.484782] [G loss: 1.231600]
[Epoch 0/20] [Batch 266/938] [D loss: 0.472888] [G loss: 0.688696]
[Epoch 0/20] [Batch 267/938] [D loss: 0.436993] [G loss: 1.094900]
[Epoch 0/20] [Batch 268/938] [D loss: 0.452706] [G loss: 0.863242]
[Epoch 0/20] [Batch 269/938] [D loss: 0.449713] [G loss: 0.861

KeyboardInterrupt: 